In [1]:
import dai
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [2]:
ed = dai.query("SELECT DISTINCT date FROM cn_stock_bar1d WHERE date >= '2024-01-01'").df()['date'].max().strftime("%Y-%m-%d")
sd = (pd.Timestamp(ed) - timedelta(days=45)).strftime("%Y-%m-%d")

In [3]:
def get_factor_df_one_instrument_one_day(instrument, date):
    
    sql = f"""
    WITH 
    cn_stock_bar5m AS(
        SELECT 
            instrument,
            DATE_TRUNC('hour', date) + INTERVAL '5 min' * CAST(FLOOR(EXTRACT(minute FROM date) / 5) AS BIGINT) AS date_time,
            make_date(year(date_time), month(date_time),  day(date_time))    AS day,
            make_time(hour(date_time), minute(date_time), second(date_time)) AS time,
            MAX(high*adjust_factor)   OVER (PARTITION BY date_time) AS high,
            MIN(low*adjust_factor)    OVER (PARTITION BY date_time) AS low,
            FIRST(open*adjust_factor) OVER (PARTITION BY date_time) AS open,
            LAST(close*adjust_factor) OVER (PARTITION BY date_time) AS close,
            SUM(volume)               OVER (PARTITION BY date_time) AS volume,
            row_number()              OVER (PARTITION BY date_time) AS _rn,
        FROM cn_stock_bar1m
        WHERE date >= '{date} 00:00:00.000' AND date <= '{date} 23:59:59.999'
        AND instrument = '{instrument}'
        QUALIFY _rn = 1
        ORDER BY date, instrument
    ),

    data1 AS (
        SELECT
            *,
            IF(ABS(close-open) <= 0.95*ABS(high-low), 1, 0) AS consist,
            IF(ABS(close-open) <= 0.95*ABS(high-low) AND close > open, 1, 0) AS consist_r,
            IF(ABS(close-open) <= 0.95*ABS(high-low) AND close < open, 1, 0) AS consist_f,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume*consist)   OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS TCV,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume*consist_r) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS PCV,
            IF(SUM(volume) OVER(PARTITION BY day, instrument) = 0, 0, SUM(volume*consist_f) OVER(PARTITION BY day, instrument) / SUM(volume) OVER(PARTITION BY day, instrument)) AS NCV,
        FROM cn_stock_bar5m
    ),

    data2 AS (
        SELECT DISTINCT 
            day AS date,
            instrument,
            TCV,
            PCV,
            NCV,
        FROM data1
    ),

    data3 AS (
        SELECT
            CAST(date AS TIMESTAMP) AS date,
            instrument,
            TCV               AS alpha_hf_00020001,
            PCV               AS alpha_hf_00020002,
            NCV               AS alpha_hf_00020003,
        FROM data2
    )

    SELECT * 
    FROM data3

    """

    data = dai.query(sql).df()
    return data

In [4]:
get_factor_df_one_instrument_one_day('600519.SH', ed)

,date,instrument,alpha_hf_00020001,alpha_hf_00020002,alpha_hf_00020003
0,2026-02-13,600519.SH,0.960136,0.557835,0.366609
